In [1]:
from IPython.display import display, Markdown
from dotenv import load_dotenv
load_dotenv()

True

# Instanciar o modelo

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio", temperature=0)

In [3]:
llm.invoke("how can langsmith help with testing?")

AIMessage(content="Langsmith, a language translation and localization platform, can assist with testing in several ways:\n\n1. **Multilingual Testing**: Langsmith allows you to test your application or website in multiple languages simultaneously, ensuring that it functions correctly across different regions and cultures.\n2. **Translation Validation**: The platform provides human-verified translations, which helps identify potential issues with translation quality, grammar, and syntax, reducing the risk of errors and improving overall user experience.\n3. **Cultural Sensitivity Testing**: Langsmith's expertise in cultural nuances can help you test your content for cultural sensitivity, ensuring that it is respectful and relevant to different audiences worldwide.\n4. **Localization Testing**: The platform offers testing services specifically designed for localization, including date, time, and number formatting, currency conversion, and other regional-specific requirements.\n5. **Autom

# inicio das chains

In [4]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world class technical documentation writer."),
    ("user", "{input}")
])

In [5]:
chain = prompt | llm 

In [6]:
chain.invoke({"input": "how can langsmith help with testing?"})

AIMessage(content="As a world-class technical documentation writer, I'd like to highlight how Langsmith can assist with testing:\n\n1. **Automated Testing**: Langsmith's AI-powered language processing capabilities can be integrated with automated testing frameworks to generate test cases and scenarios in multiple languages. This ensures that your software or application is thoroughly tested across different linguistic and cultural contexts.\n2. **Test Data Generation**: Langsmith can help generate test data, such as user input, error messages, and validation rules, in various languages. This enables you to test your application's language-specific features and edge cases more effectively.\n3. **Localization Testing**: Langsmith's machine translation capabilities can facilitate localization testing by generating translated versions of your software or application. This allows you to test the localized version with native speakers, ensuring that it meets cultural and linguistic requireme

# output parser

In [7]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [8]:
chain = prompt | llm | output_parser

In [9]:
chain.invoke({"input": "how can langsmith help with testing?"})

"As a world-class technical documentation writer, I'd like to highlight how Langsmith can assist with testing:\n\n1. **Automated Testing**: Langsmith's AI-powered language processing capabilities can be integrated with automated testing frameworks to generate test cases and scenarios in multiple languages. This ensures that your software or application is thoroughly tested across different linguistic and cultural contexts.\n2. **Test Data Generation**: Langsmith can help generate test data, such as user input, error messages, and validation rules, in various languages. This enables you to test your application's language-specific features and edge cases more effectively.\n3. **Localization Testing**: Langsmith's machine translation capabilities can facilitate localization testing by providing translated versions of your software or application for testing purposes. This helps identify any issues related to language, culture, or region-specific requirements.\n4. **Error Message Translat

# RAG

In [10]:
# Make sure to `pip install openai` first
from openai import OpenAI
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

def get_embedding(text, model="CompendiumLabs/bge-large-en-v1.5-gguf"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

print(get_embedding("Once upon a time, there was a cat."))

[0.032019928097724915, 0.06339229643344879, -0.14854365587234497, -0.0749576985836029, 0.05321422964334488, 0.0338364876806736, -0.08441770821809769, 0.040955618023872375, -0.022981032729148865, 0.006492594722658396, -0.002992316149175167, 0.03711792826652527, 0.07530311495065689, 0.03950294107198715, -0.07141084969043732, -0.07568350434303284, 0.03580918163061142, -0.035198185592889786, 0.023276833817362785, 0.05217314511537552, -0.0002876944199670106, -0.03401849791407585, 0.06201925501227379, 0.027593346312642097, 0.03676736727356911, 0.05155615136027336, -0.047059908509254456, 0.04728643223643303, -0.013932494446635246, 0.011554342694580555, 0.019040672108530998, -0.025935795158147812, 0.006764427293092012, -0.04029737412929535, -0.016108892858028412, -0.03797941654920578, 0.07615499198436737, 0.014955293387174606, 0.04871508851647377, 0.057611603289842606, 0.028666773810982704, 0.038383256644010544, -0.041735850274562836, -0.00832691416144371, 0.049295950680971146, 0.0231716521084

In [11]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/tutorials/evaluation")

docs = loader.load()

In [12]:
from langchain_core.embeddings import Embeddings
from typing import List

class CustomEmbeddings(Embeddings):
    """Interface for embedding models."""
    base_url="http://localhost:1234/v1"
    api_key="lm-studio"
    embed_model="CompendiumLabs/bge-large-en-v1.5-gguf"
    client = None

    def create_client(self):
        self.client = OpenAI(base_url=self.base_url, api_key=self.api_key)

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """Embed search docs."""
        if not self.client:
            self.create_client()
        return [self.client.embeddings.create(input = [text], model=self.embed_model).data[0].embedding for text in texts]

    def embed_query(self, text: str) -> List[float]:
        """Embed query text."""
        if not self.client:
            self.create_client()
        return self.client.embeddings.create(input = [text], model=self.embed_model).data[0].embedding


In [13]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, CustomEmbeddings())


In [14]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [15]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

'According to the provided context, Langsmith can help with testing by allowing users to "visualize test results".'

In [16]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [17]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
display(Markdown(response["answer"]))

According to the provided context, LangSmith can help with testing by allowing users to:

1. Create a dataset: LangSmith allows users to define a dataset with inputs and expected outputs, which can be used to evaluate their application.
2. Define metrics: Users can define metrics to evaluate their application's performance, such as correctness and length of the output.
3. Run evaluations: LangSmith provides an evaluation framework that allows users to run their application on a dataset and evaluate its performance using defined metrics.
4. Compare results: LangSmith enables users to compare the results of different runs by displaying them in the Experiments tab.

By using LangSmith, developers can easily test and evaluate their applications, identify areas for improvement, and optimize their models for better performance.

# Conversation Retrieval Chain

In [18]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [19]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

[Document(page_content='Evaluate your LLM application | 🦜️🛠️ LangSmith', metadata={'source': 'https://docs.smith.langchain.com/tutorials/evaluation', 'title': 'Evaluate your LLM application | 🦜️🛠️ LangSmith', 'description': 'It can be hard to measure the performance of your application with respect to criteria important you or your users.', 'language': 'en'}),
 Document(page_content="Skip to main contentLangSmith API DocsSearchGo to AppQuick startTutorialsAdd observability to your LLM applicationEvaluate your LLM applicationOptimize a classifierHow-to guidesConceptsReferencePricingSelf-hostingTutorialsEvaluate your LLM applicationOn this pageEvaluate your LLM applicationIt can be hard to measure the performance of your application with respect to criteria important you or your users.\nHowever, doing so is crucial, especially as you iterate on your application.\nIn this guide we will go over how to test and evaluate your application.\nThis allows you to measure how well your application

In [20]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [21]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(page_content='Evaluate your LLM application | 🦜️🛠️ LangSmith', metadata={'source': 'https://docs.smith.langchain.com/tutorials/evaluation', 'title': 'Evaluate your LLM application | 🦜️🛠️ LangSmith', 'description': 'It can be hard to measure the performance of your application with respect to criteria important you or your users.', 'language': 'en'}),
  Document(page_content="Skip to main contentLangSmith API DocsSearchGo to AppQuick startTutorialsAdd observability to your LLM applicationEvaluate your LLM applicationOptimize a classifierHow-to guidesConceptsReferencePricingSelf-hostingTutorialsEvaluate your LLM applicationOn this pageEvaluate your LLM applicationIt can be hard to measure the performance of your application with respect to criteria important you or your users.\nHowever, doing so is crucial, especially as you 

# Agent

In [34]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [35]:
from langchain_community.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

In [25]:
tools = [retriever_tool, search]

In [26]:
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")

# You need to set OPENAI_API_KEY environment variable or pass it as argument `api_key`.
llm = ChatOpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio", temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [27]:
agent_executor.invoke({"input": "how can langsmith help with testing?"})    




> Entering new AgentExecutor chain...
Langsmith is a powerful tool that can assist with testing in several ways:

1. **Automated Testing**: Langsmith can be used to generate test cases and automate the testing process. It can create test data, execute tests, and report results.
2. **Code Review**: Langsmith's AI-powered code analysis capabilities can help identify potential issues, such as syntax errors, logical flaws, or security vulnerabilities, in your code before it's even executed.
3. **Test Data Generation**: Langsmith can generate test data that is relevant to the specific requirements of your application, making it easier to create comprehensive tests.
4. **Test Case Prioritization**: By analyzing the complexity and criticality of different test cases, Langsmith can help you prioritize which tests to run first, ensuring that the most important ones are executed.
5. **Regression Testing**: Langsmith's ability to analyze code changes and identify potential regressions can help 

{'input': 'how can langsmith help with testing?',
 'output': "Langsmith is a powerful tool that can assist with testing in several ways:\n\n1. **Automated Testing**: Langsmith can be used to generate test cases and automate the testing process. It can create test data, execute tests, and report results.\n2. **Code Review**: Langsmith's AI-powered code analysis capabilities can help identify potential issues, such as syntax errors, logical flaws, or security vulnerabilities, in your code before it's even executed.\n3. **Test Data Generation**: Langsmith can generate test data that is relevant to the specific requirements of your application, making it easier to create comprehensive tests.\n4. **Test Case Prioritization**: By analyzing the complexity and criticality of different test cases, Langsmith can help you prioritize which tests to run first, ensuring that the most important ones are executed.\n5. **Regression Testing**: Langsmith's ability to analyze code changes and identify pot

In [28]:
agent_executor.invoke({"input": "what is the weather in SF?"})



> Entering new AgentExecutor chain...
According to current weather reports, San Francisco (SFO) has:

**Current Weather:** Partly Cloudy

**Temperature:** 58°F (14°C)

**Humidity:** 64%

**Wind:** Light breeze at 7 mph (11 km/h)

**UV Index:** Moderate (5 out of 10)

Please note that weather conditions can change rapidly, so it's always a good idea to check the latest forecast before heading out. Would you like me to provide more information or help with anything else?

> Finished chain.


{'input': 'what is the weather in SF?',
 'output': "According to current weather reports, San Francisco (SFO) has:\n\n**Current Weather:** Partly Cloudy\n\n**Temperature:** 58°F (14°C)\n\n**Humidity:** 64%\n\n**Wind:** Light breeze at 7 mph (11 km/h)\n\n**UV Index:** Moderate (5 out of 10)\n\nPlease note that weather conditions can change rapidly, so it's always a good idea to check the latest forecast before heading out. Would you like me to provide more information or help with anything else?"}

In [31]:
search.run("what is the weather in SF?")


RatelimitException: https://links.duckduckgo.com/d.js?q=what+is+the+weather+in+SF%3F&kl=wt-wt&l=wt-wt&p=&s=0&df=y&vqd=4-304725744095816561580234963354421099979&ex=-1 202 Ratelimit